In [ ]:
import models
import torch
from torchvision import transforms
from util.path_functions import PathFunctions
from PIL import Image
import os
from util.visualization import Visualization


FILENAME = "mnist-vit"
WEIGHTPATH = f"dev/result/weight/{FILENAME}.pth"
SAVEPDFPATH = f"dev/result/img/{FILENAME}-attack.pdf"
RESEIZE=224

In [ ]:
def evaluate(model, image):
  with torch.no_grad():
    output = model(image)
  return output

In [ ]:
model = models.vit_tiny_16_224(pretrained=False, num_classes=10, img_size=RESEIZE)
checkpoint = torch.load(PathFunctions.absolutePath(WEIGHTPATH))
model.load_state_dict(checkpoint)
model.eval()

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # 1チャンネルを3チャンネルに変換
    transforms.Resize((RESEIZE, RESEIZE)),  # ViTモデルは大きな画像が必要
    transforms.ToTensor()
])

In [ ]:
# # フォルダのパス
# folder_path = "/datasets/mnist/Image/attack/0/"

# # フォルダ内のファイル一覧を取得
# files = PathFunctions.getFileList(folder_path)
# print(files)

# for file in files:
#     file = PathFunctions.absolutePath(os.path.join(folder_path + file))
#     image = Image.open(file).convert('RGB')
#     image = transform(image)
#     image = image.unsqueeze(0)
#     output = evaluate(model=model, image=image)
#     print(output)

## Black Box Attacks

In [ ]:
for i in range(1):
    folder_path = f"/datasets/mnist/Image/attack/overlay/{i}/"
    files = PathFunctions.getFileList(folder_path)
    for file in files:
        file = PathFunctions.absolutePath(os.path.join(folder_path + file))
        image = Image.open(file).convert('RGB')
        image = transform(image)
        image = image.unsqueeze(0)
        output = evaluate(model=model, image=image)
        output_np = output.numpy()


In [ ]:
# # フォルダのパス
# folder_path = "/datasets/mnist/Image/attack/0/"

# # フォルダ内のファイル一覧を取得
# files = PathFunctions.getFileList(folder_path)
# print(files)

# for file in files:
#     file = PathFunctions.absolutePath(os.path.join(folder_path + file))
#     image = Image.open(file).convert('RGB')
#     image = transform(image)
#     image = image.unsqueeze(0)
#     output = evaluate(model=model, image=image)
#     print(output)

In [ ]:
print("-- Active Attention Map")
Visualization_instance = Visualization()
model = models.vit_tiny_16_224(pretrained=False, num_classes=10)

# List to store the results for each image

for i in range(10):
    folder_path = f"/datasets/mnist/Image/attack/overlay/{i}/"
    files = PathFunctions.getFileList(folder_path)
    files_per_group = 6
    files_groups = [files[i:i + files_per_group] for i in range(0, len(files), files_per_group)]
    for chnageNum, files_group in enumerate(files_groups):
        imagesList = []
        for file in files_group:
            file = os.path.join(folder_path + file)
            results = Visualization_instance.attention_map(model=model, checkpointPath=WEIGHTPATH, imagePath=file)
            # Append the results to the images list
            imagesList.append(results)
        Visualization_instance.show_attention_map(imagesList=imagesList, pdfPath=SAVEPDFPATH+f"{i}_"+f"{chnageNum}.pdf")
